# This implementation of the genetic algorithm randomly samples shortfall missions and bins them

This is used to find the mean and standard deviatioin of the possible missions

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [ ]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime
from scipy.stats import genpareto
from sympy import *

# Loads in the analog payloads and the shortfalls scores

In [12]:
#import cell

#tech
tech_array = []

#take data from tech csv and convert it to an array
with open('Found_Technologies.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_array.append(line)


#take data from scores tech csv and convert it to an array
tech_score = []
with open('Shortfalls_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        tech_score.append(line)

#print(tech_score)



# Prepares data from the .csv files and sets parameter values

In [ ]:
#data processing cell

#Parse shortfall data 
tech_array_mod          = tech_array[1:]                                    #cut off title score
tech_mass_array         = np.asarray([i[-1] for i in tech_array_mod],float)      #Masses of techs 0 indexed
tech_power_array        = np.asarray([i[-2] for i in tech_array_mod],float)      #power consumption of techs 0 indexed
shortfall_array_temp    = [np.array(i[1:-2]) for i in tech_array_mod]       
shortfall_array         = []
for i in shortfall_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    shortfall_array.append(arr)

shortfall           = np.asarray(tech_score[1:],float)
shortfall_matrix    = np.zeros((len(shortfall_array),len(shortfall)))
for i in range(len(shortfall_matrix)):
    for j in shortfall_array[i]:
        shortfall_matrix[i][j-1] = 1

shortfall_matrix = np.transpose(shortfall_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

shortfall_integrated    = shortfall[0:, 1]  #Scores from NASA 0 indexed
shortfall_consequence   = shortfall[0:, 2]  #Scores from consequence axis 0 indexed
shortfall_liklihood     = shortfall[0:, 3]  #Scores from liklihood axis 0 indexed

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function

In [14]:
#score function for tech

def score_tech(mission, check_mass=True):
    #weights
    w1 = 2.3  #completed_score
    w2 = 7  #integrated_score
    w3 = 4  #consequence_score
    w4 = 4  #liklihood_score

    mass_max_tech = max_mass*(1-mass_ratio) #in kg
    mass_min_tech = min_mass*(1-mass_ratio) #in kg

    #Mass check
    if check_mass and (tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) <= mass_min_tech or tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission) > mass_max_tech):
        return abs(tech_mass_array.dot(mission) + beta*tech_power_array.dot(mission))

    #normlizers
    max_number      = 93        #normalize number score 0-1 
    max_integrated  = 8.1035    #normalize integrated score 0-1
    max_consequence = 5         #normlize consquence score 0-1
    max_liklihood   = 5         #normlize consquence score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3+w4 
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W
    w4 = 10*w4/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(shortfall_matrix @ mission, a_min=None, a_max=1) #A shortfall is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of shortfalls completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_integrated_score  = shortfall_integrated  @ normalized_completed             #Generates integrated score
    completed_consequence_score = shortfall_consequence @ normalized_completed             #Generates the consquences score
    completed_liklihood_score   = shortfall_liklihood   @ normalized_completed             #Generates the liklihood score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_integrated_score/max_integrated + w3*completed_consequence_score/max_consequence + w4*completed_liklihood_score/max_liklihood
    return -score


# Setup the number of bins

In [29]:
#setup
num_bins = 300
bins = np.zeros(num_bins)
bin_val = np.array([(10/num_bins) + 10*i/num_bins for i in range(num_bins)])
legth = len(tech_array_mod)

# Loop through random comboniations

In [ ]:
#loop 3*10^6 10min 33s
for i in range(int(36*10**6)):
    mission = np.random.choice([0, 1], size=legth, p=[.8, .2])
    cur = -score_tech(mission)
    for i in range(len(bin_val)-1):
        if cur <= bin_val[i] and cur > bin_val[i-1]:
            bins[i] += 1


# Save to a .csv file

In [37]:
#csv
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
#write to csv
with open("bins_tech" + formatted_time + '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    wr.writerow(bin_val)
    wr.writerow(bins)